In [4]:
from selenium import webdriver
import time

In [5]:
# open chrome browser
browser = webdriver.Chrome(executable_path='../../../../chromedriver.exe')

In [6]:
# url in address window(주소창에 url 입력)
browser.get('https://play.google.com/store/search?q=%ED%97%AC%EC%8A%A4%EC%BC%80%EC%96%B4%EC%95%B1&c=apps&hl=ko-KR')

In [7]:
browser.implicitly_wait(10)

## 리뷰 내용 가져오기
- 리뷰 묶음 tag : div.fysCi > div > div:nth-child(2) > div
- 리뷰 내용 tag : div.h3YV2d
- 리뷰 등록 날짜 tag :  header > div.Jx4nYe > span
- 별점 tag : header > div.Jx4nYe > div

In [8]:
import pandas as pd
import pymongo as mg

In [10]:
client = mg.MongoClient(host='mongodb://localhost:27017')
database = client['study_data_analytics']
collection = database['googlestore_heathcare']

# 서비스 앱에 대한 리스트 가져오기
click_path = 'section > div > div > div > div'
company_list = browser.find_elements_by_css_selector(click_path)
len(company_list)

# 특정 서비스 클릭
for company in company_list:
    company.click()
    time.sleep(2)
    
    # 서비스 앱에 대한 정보 가져오기
    # ** 개인이 만든 앱 경우 company_name없을 수도 있음 -> 각 변수값이 없을 수 있기 때문에 그런 경우는 공백으로 채워줘야 함
    # ** 각 변수들을 try문으로 감싸줌
    try :
        service_name = browser.find_element_by_css_selector('div > h1 > span').text
    except :
        service_name = str()
    try :
        company_name = browser.find_element_by_css_selector('div.Il7kR > div > div > div > a > span').text
    except :
        company_name = str()
    try : 
        app_information = browser.find_element_by_css_selector('div:nth-child(1) > c-wiz:nth-child(2) > div > section > div').text
    except :
        app_information = str()
    # [service_name, company_name, app_information]

    #댓글 팝업 띄우기(리뷰가 없을 시 대비한 방어 코드까지 작성함)
    review_flag = True
    try :
        browser.find_element_by_xpath('//*/div/div/button/*[contains(text(), "리뷰")]').click()
    except :
        review_flag = False
    
    #회사 서비스에 대한 전체 리뷰 가져오기
    serviceapp_columns_name = ['service_name', 'content', 'date', 'stars']
    reviews_list = list()
    if review_flag :
        reviews_bundle = browser.find_elements_by_css_selector('div.fysCi > div > div:nth-child(2) > div')
        len(reviews_bundle)

        for review_bundle in reviews_bundle:
            content = review_bundle.find_element_by_css_selector('div.h3YV2d').text
            date = review_bundle.find_element_by_css_selector('header > div.Jx4nYe > span').text
            stars = review_bundle.find_element_by_css_selector('header > div.Jx4nYe > div').get_attribute('aria-label')
            review_list = [service_name, content, date, stars]
            reviews_list.append(review_list) # 특정 회사 서비스의 리뷰에 대한 모든 것이 담긴다
        
        browser.back() #리뷰 팝업 창 닫기
        
        # 저장 위한 작업
        df_reviews = pd.DataFrame(data=reviews_list, columns=serviceapp_columns_name) # 저장을 위한 작업
        data_dict = df_reviews.to_dict(orient='records')
        collection.insert_many(data_dict)
        
    print('service_name : {}, review count : {}'.format(service_name, len(reviews_list))) # 서비스 이름과 그 서비스에 달린 댓글 개수 확인용
    browser.back() #리스트로 돌아가기
    time.sleep(3)
print('Done')

service_name : Samsung Health(삼성 헬스), review count : 40
service_name : NH헬스케어, review count : 40
service_name : , review count : 0
service_name : , review count : 0
service_name : SK VIEW 헬스케어, review count : 2
service_name : 애니핏 플러스 나에게 핏한 건강관리, review count : 40
service_name : 헬스케어 건강관리서비스, review count : 1
service_name : 오케어, review count : 40
service_name : 채움건강, review count : 40
Done


In [ ]:
# browser.quit()